# Value-at-Risk for Stocks: Delta-Normal Approach

### Lecture Notes by Jakov Ivan S. Dumbrique (jdumbrique@ateneo.edu)

MA 195L.2: Introduction to Financial Mathematics II \
Second Semester, S.Y. 2020-2021 \
Ateneo de Manila University

In [1]:
import numpy as np # Numerical Computing
import pandas as pd # Data wrangling
from statistics import NormalDist # statistical analysis

In [2]:
def get_return(df):
    return np.log(df["close"]/df["previous"])

def one_day_alpha_percent_VaR_single_stock_delta_normal(df, N, alpha):
    
    """Returns the one-day 100(alpha)% VaR of a single stock using Delta Normal Approach.
    
    Parameters
    ----------
    df : pandas.DataFrame
        has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row  
    N : int
        number of shares for the sole stock
    alpha : int
        the value to be used in calculting the 100(alpha)% VaR (e.g. 0.99, 0.95)
    
    Returns
    -------
    float (2 decimal places)
        one-day 100(alpha)% VaR of a single stock using Delta Normal Approach
    """

    S_0 = df.loc[0, "close"]
    
    # creates a new column that shifts the prices d values up (or -d values down)
    df["previous"] = df["close"].shift(-1)
    # get the daily returns
    df["return"] = df.apply(get_return, axis=1)
    
    # standard deviation of your stock returns
    sigma = df["return"].std()
    
    quantile = NormalDist().inv_cdf(alpha)
    var = N * S_0 * sigma * quantile
    
    return round(var, 2)
    

## VaR for Stocks: Two-Asset Portfolio

Consider a portfolio containing $N_1$ and $N_2$ shares of two stocks with current price $S_{1,0}$ and $S_{2,0}$. Let $\Delta P$ denote the change in portfolio value, and $\mathbf{\alpha}$ be the vector of asset weights

\begin{equation}
\mathbf{\alpha}=\left[\begin{array}{cc}
					N_1 S_{1,0} & N_2 S_{2,0}
					\end{array}\right]
\end{equation}

We assume that $\mathbf{R} = \left[\begin{array}{c}
					R_1 \\ R_2
					\end{array}\right]$ has a bivariate normal distribution with mean $\mathbf{0}$ and covariance matrix $\mathbf{\Sigma}$.

It follows from the property of the bivariate normal distribution that $\Delta P\sim N(0,\sigma_P^2)$, where

\begin{equation}
\sigma_P^2 = \mathbf{\alpha}^T\mathbf{\Sigma\alpha}.
\end{equation}

The **99\% diversified portfolio VaR** is given by
\begin{equation}
|V_d| =\sigma_P\Phi^{-1}(0.99).
\end{equation}

## Exercise 1:
Today is February 23, 2018. You are a portfolio risk manager who is assigned to analyze the market risk of a portfolio of 700 PLDT (TEL) shares and 300 Ayala Corporation (AC) shares. What is the portfolio's one-day 95% diversified VaR using the delta-normal approach? 

In [4]:
TEL_df = pd.read_csv('../data/TEL_2018.csv')
AC_df = pd.read_csv('../data/AC_2018.csv')

In [5]:
TEL_df

,dt,close
0,2/23/18,1488.74
1,2/22/18,1510.86
2,2/21/18,1513.72
3,2/20/18,1536.65
4,2/16/18,1476.37
...,...,...
243,3/2/17,1430.29
244,3/1/17,1408.94
245,2/28/17,1394.35
246,2/27/17,1374.93


In [6]:
AC_df

,dt,close
0,2/23/18,926.04
1,2/22/18,926.77
2,2/21/18,929.44
3,2/20/18,933.04
4,2/16/18,934.30
...,...,...
243,3/2/17,779.22
244,3/1/17,778.47
245,2/28/17,775.90
246,2/27/17,772.54


In [9]:
N1 = 700
N2 = 300
S1 = TEL_df.loc[0, "close"]
S2 = AC_df.loc[0, "close"]

### Step 1: Constructing the vector of asset weights $\mathbf{\alpha}$
\begin{equation}
\mathbf{\alpha}=\left[\begin{array}{cc}
					N_1 S_{1,0} & N_2 S_{2,0}
					\end{array}\right]
\end{equation}

In [23]:
alpha_vector = [N1*S1, N2*S2]
alpha_vector = np.array(alpha_vector)
alpha_vector

array([1042118.,  277812.])

### Step 2: Calculating the covariance matrix $\mathbf{\Sigma}$ of the Return vector $\mathbf{R}$

#### Step 2.1: Getting the historical return vectors

In [13]:
# creates a new column that shifts the prices d values up (or -d values down)
TEL_df["previous"] = TEL_df["close"].shift(-1)
AC_df["previous"] = AC_df["close"].shift(-1)

R1_vector = TEL_df.apply(get_return, axis=1)
R2_vector = AC_df.apply(get_return, axis=1)

In [15]:
R2_vector

0     -0.000788
1     -0.002877
2     -0.003866
3     -0.001350
4     -0.001935
         ...   
243    0.000963
244    0.003307
245    0.004340
246    0.002047
247         NaN
Length: 248, dtype: float64

#### Step 2.2: Calculating the covariance matrix $\mathbf{\Sigma}$

In [32]:
cov_matrix = np.cov([R1_vector[:-1], R2_vector[:-1]])
cov_matrix

array([[3.85307884e-04, 1.23496261e-05],
       [1.23496261e-05, 3.06084441e-04]])

### Step 3: Computing for the standard deviation $\sigma_P$ of $\Delta P$
\begin{equation}
\sigma_P^2 = \mathbf{\alpha}^T\mathbf{\Sigma\alpha}.
\end{equation}

In [33]:
sigma = np.sqrt(np.dot(alpha_vector.T, np.dot(cov_matrix, alpha_vector)))
sigma

21194.866931340945

### Step 4: Calculating for the Value-at-Risk

In [34]:
alpha = 0.95
var = sigma * NormalDist().inv_cdf(alpha)
round(var, 2)

34862.45